In [1]:
import pandas as pd
import re

In [2]:
# Devulve una lista con los usernames mencionados en el tweet
def getMentions(row):
    row.loc['tweet'] = re.findall(r'[@]\w+', row.loc['tweet'])
    return row

In [3]:
# Elimina los @ del target
def cleanTarget(row):
    row.loc['target'] = row.loc['target'][1:]
    return row

In [4]:
# Obtiene una lista con los nombres de usuario de los concejales
lista_concejales = pd.read_csv('concejales.csv')['username'].tolist()

In [9]:
df = pd.read_csv('tweets_concejales.csv')

# Cogemos las columnas que nos interesan
df = df[['id','date','time','username','tweet']]

# Une las columnas date y time en una sola
df['start'] = pd.to_datetime(df['date'] + ' ' +  df['time'])

df.drop(['date', 'time'], axis=1, inplace=True)

# Elimina los tweets de antes del inicio de la actual legislatura (twint no lo hace bien)
df = df[df['start'] > '2019-06-15']

# Borramos duplicados
df.drop_duplicates(subset=None, keep='first', inplace=True)
df.drop(['id'], axis=1, inplace=True)

# Saca los usuarios mencionados del texto de los tweets
df = df.apply(getMentions, axis=1)

# Cambia nombres de columnas
df.rename(columns={"username": "source", "tweet": "target"}, inplace=True)

# Explotamos los targets para que cada fila corresponda con una interación
df = df.explode('target',ignore_index=True)

# Quita el @ de los target
df = df.apply(cleanTarget, axis=1)

# Quita las interacciones con personas que no son de la red
df = df[df["target"].isin(lista_concejales)]

# Pasa todos los target a minúscula
df['target'] = df['target'].str.lower()

In [6]:
df.to_csv('grafo_concejales.csv', index=False)